# Interannual variability intercomparison between Landsat and MODIS


In [ ]:
import numpy as np
import xarray as xr
import seaborn as sb
import pandas as pd
import geopandas as gpd
import matplotlib as mpl
from matplotlib.cm import ScalarMappable
import contextily as ctx
import matplotlib.pyplot as plt
from odc.geo.xr import assign_crs
from matplotlib.ticker import FormatStrFormatter

import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords

# Using ggplot styles in this notebook
plt.style.use('ggplot')

%matplotlib inline

## Analysis Parameters

In [ ]:
model_var='NDVI'
crs='epsg:4326'
base = '/g/data/os22/chad_tmp/AusENDVI/data/'

## Open datasets

In [ ]:
modis = xr.open_dataset(base+model_var+'_harmonization/MODIS_'+model_var+'_5km_monthly_200003_202212.nc')[model_var+'_median']
modis = assign_crs(modis, crs=crs)
modis.attrs['nodata'] = np.nan

ls = xr.open_dataset(base+'/NDVI_harmonization/Landsat_NDVI_5km_monthly_1988_2012.nc')['NDVI']
ls = assign_crs(ls, crs=crs)
ls.attrs['nodata'] = np.nan

merge = xr.open_dataset(f'{base}/NDVI_harmonization/LGBM/NDVI_CLIM_LGBM_5km_monthly_1982_2022_noGaps.nc')['NDVI']
merge = assign_crs(merge, crs=crs)
merge.attrs['nodata'] = np.nan
merge = merge.rename('NDVI')

rain = xr.open_dataset('/g/data/os22/chad_tmp/AusENDVI/data/5km/rain_5km_monthly_1981_2022.nc').rain
rain = assign_crs(rain, crs=crs)
rain.attrs['nodata'] = np.nan

## Match datasets

In [ ]:
ls_trimmed = ls.sel(time=slice('2000-03', '2012'))
modis_trimmed = modis.sel(time=slice('2000-03', '2012'))

#create nodata masks
m1 =  ~np.isnan(ls_trimmed)
m2 =  ~np.isnan(modis_trimmed)

#combine masks
mask = (m1 & m2)

ls_trimmed = ls_trimmed.where(mask)
modis_trimmed = modis_trimmed.where(mask)

## Calculate standardised anomalies

In [ ]:
import warnings
warnings.simplefilter('ignore')

#standardized anom
def stand_anomalies(ds):
    return xr.apply_ufunc(
        lambda x, m, s: (x - m) / s,
            ds.groupby("time.month"),
            ds.groupby("time.month").mean(),
            ds.groupby("time.month").std()
    )
    
ls_std_anom = stand_anomalies(ls_trimmed)
modis_std_anom = stand_anomalies(modis_trimmed)
rain_std_anom = stand_anomalies(rain)

## Rolling mean anomalies

In [ ]:
roll=12

In [ ]:
rain_df = rain_std_anom.rename('rain').rolling(time=roll,
                min_periods=roll).mean().mean(['longitude','latitude']).sel(time=slice('2000', '2013')).to_dataframe().drop(['spatial_ref', 'month'], axis=1)

plt.style.use('default')
fig, ax = plt.subplots(1,1, figsize=(14,4))
ax2 = ax.twinx()

ls_std_anom.drop('month').rolling(time=roll, min_periods=roll).mean().mean(['longitude','latitude']).plot(ax=ax, label='Landsat (DEA-NBAR)')
modis_std_anom.drop('month').rolling(time=roll, min_periods=roll).mean().mean(['longitude','latitude']).plot(ax=ax, label='MODIS MCD43A4')

norm=plt.Normalize(-2.5,2.5)
cmap = mpl.colors.LinearSegmentedColormap.from_list("", ['saddlebrown','chocolate','white','darkturquoise','darkcyan'], N=256)

# Plot bars
bar = ax2.bar(rain_df.index, 1, color=cmap(norm(rain_df['rain'])), width=32)
sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax2, shrink=0.8, pad=0.01)
cbar.set_label('Rainfall Anomaly (z-score)',labelpad=.5)

ax2.set_zorder(ax.get_zorder()-1)
ax.set_frame_on(False)
ax.axhline(0, c='grey', linestyle='--')

# Reformat y-axis label and tick labels
ax.set_ylabel(model_var+' Anomaly (z-score)')
ax.set_xlabel('')
ax2.set_ylabel('')
ax2.set_yticks([])
ax2.set_ylim([0, 1]) 
ax.margins(x=0)
ax2.margins(x=0)

# Adjust the margins around the plot area
plt.subplots_adjust(left=0.1, right=None, top=None, bottom=0.2, wspace=None, hspace=None)

ax.legend()
ax.set_title(None);
fig.savefig("/g/data/os22/chad_tmp/AusENDVI/results/figs/MODIS_landsat_anomalies_"+str(roll)+"Mrollingmean.png",
            bbox_inches='tight', dpi=300)